# Animate GPX tracks on the map

The goal is to help aligning a raster orienteering map with GPS, align GPX tracks temporally and display them as a head-to-head race.

In [ ]:
from IPython.display import display, FileLink
import ipywidgets as widgets

import sys
sys.path.append("./src")
from app import App

app = App()

## 1. Select a couple of GPX tracks

The working are will be determined from the first track.

In [ ]:
gpx_upload = widgets.FileUpload(accept='.gpx', multiple=True)
display(gpx_upload)

#@debounce(0.2)
def on_tracks_value_change(change):
    app.set_tracks([file['content'] for file in gpx_upload.value.values()])

gpx_upload.observe(on_tracks_value_change, names='value')

## 2. Now upload a raster image of the orienteering map


Select features on the image as far from each other as possible.
Then mark the same features on the map.

If this step is omitted, there will be no overlay.

In [ ]:
img_upload = widgets.FileUpload(accept='image/*', multiple=False)
display(img_upload, app.anchors_out)

#@debounce(0.2)
def on_img_value_change(change):
    img = next(iter(img_upload.value.values()), None)
    if img:
        app.set_image(img['content'])

img_upload.observe(on_img_value_change, names='value')

## 3. Upload some positional images

In [ ]:
pics_upload = widgets.FileUpload(accept='image/*', multiple=True)
display(pics_upload)

def on_pics_value_change(change):
    app.set_pictures([i['content'] for i in pics_upload.value.values()])

pics_upload.observe(on_pics_value_change, names='value')

## 4. Get the map with the aligned image map and track race

In [ ]:
display(app.map_out)